# Chapter 3: Spatial data_path_path operations

## Prerequisites

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.ndimage
import scipy.stats
import shapely
import geopandas as gpd
import contextily as cx
import rasterio
import rasterio.plot
import rasterio.merge
import rasterio.features

In [ ]:
data_path = "F:\\books\\geocompy-main\\data\\"
output_path = "F:\\books\\geocompy-main\\output\\"

nz = gpd.read_file(data_path + 'nz.gpkg')
nz_height = gpd.read_file(data_path + 'nz_height.gpkg')
world = gpd.read_file(data_path + 'world.gpkg')
cycle_hire = gpd.read_file(data_path + 'cycle_hire.gpkg')
cycle_hire_osm = gpd.read_file(data_path + 'cycle_hire_osm.gpkg')
src_elev = rasterio.open(output_path + 'elev.tif')
src_landsat = rasterio.open(data_path + 'landsat.tif')
src_grain = rasterio.open(output_path + 'grain.tif')

## 3.1 Introduction

## 3.2 Spatial Operations on vector data

### 3.2.1 Spatial subsetting

In [ ]:
canterbury = nz[nz.Name == 'Canterbury']
canterbury.plot()

In [ ]:
canterbury_shape = canterbury.geometry.iloc[0]
canterbury_shape

In [ ]:
sel = nz_height.intersects(canterbury_shape)
sel

In [ ]:
canterbury_height = nz_height[sel]
canterbury_height

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax = canterbury.plot(
    ax=ax,
    color = 'lightgrey',
    edgecolor = 'black'
)
canterbury_height.plot(
    ax=ax,
    color = 'red',
    edgecolor = 'red'
)

In [ ]:
base = nz.plot(color='white', edgecolor='lightgray')
nz_height.plot(ax=base, color='lightblue', edgecolor='black')

In [ ]:
base = nz.plot(color='white', edgecolor='lightgray')
canterbury.plot(ax=base, color='lightblue', edgecolor='black')
canterbury_height.plot(ax=base, color='lightblue', edgecolor='black')

In [ ]:
canterbury_southland = nz[nz.Name.isin(['Canterbury', 'Southland'])]
sel = nz_height.intersects(canterbury_southland.union_all())
canterbury_southland_height = nz_height[sel]
canterbury_southland_height

In [ ]:
base = nz.plot(color='white', edgecolor='lightgray')
nz_height.plot(ax=base, color='white', edgecolor='red')

In [ ]:
base = nz.plot(color='white', edgecolor='lightgray')
canterbury_southland.plot(ax=base, color='lightgrey', edgecolor='black')
canterbury_southland_height.plot(ax=base, color='none', edgecolor='red')

In [ ]:
m = canterbury_southland.explore(
    tiles='Esri WorldImagery',
    color = 'green',
)
canterbury_southland_height.explore(
    m=m,
    color = 'green',
    style_kwds = {'color': 'red', 'opacity': 0.5, 'fillOpacity': 0.1},
    marker_kwds = {'radius': 5}
)

### 3.2.2 Topological relations

In [ ]:
points = gpd.GeoSeries([
  shapely.Point(0.2,0.1), 
  shapely.Point(0.7,0.2), 
  shapely.Point(0.4,0.8)
])
line = gpd.GeoSeries([
  shapely.LineString([(0.4,0.2), (1,0.5)])
])
poly = gpd.GeoSeries([
  shapely.Polygon([(0,0), (0,1), (1,1), (1,0.5), (0,0)])
])

In [ ]:
list(enumerate(points))

In [ ]:
base = poly.plot(color='lightgrey', edgecolor='red')
line.plot(ax=base, color='black', linewidth=7)
points.plot(ax=base, color='none', edgecolor='black')
for i in enumerate(points):
    base.annotate(
        i[0], xy=(i[1].x, i[1].y),
        xytext=(3, 3),
        textcoords="offset points",
        weight='bold'
    )

In [ ]:
points.intersects(poly.iloc[0])

In [ ]:
poly2 = gpd.GeoSeries([
  shapely.Polygon([(0,0), (0,1), (1,1), (1,0.5), (0,0)]),
  shapely.Polygon([(0,0), (1,0.5), (1,0), (0,0)])
])

In [ ]:
base = poly2.plot(color='lightgrey', edgecolor='red')
points.plot(ax=base, color='none', edgecolor='black')
for i in enumerate(points):
    base.annotate(
        i[0], xy=(i[1].x, i[1].y),
        xytext=(3, 3),
        textcoords="offset points",
        weight='bold'
    )

In [ ]:
points.apply(lambda x: poly2.intersects(x))

In [ ]:
points.apply(lambda x: poly2.intersects(x)).to_numpy()

In [ ]:
points.within(poly.iloc[0])

In [ ]:
points.touches(poly.iloc[0])

In [ ]:
points.disjoint(poly.iloc[0])

In [ ]:
points.distance(poly.iloc[0]) < 0.2

In [ ]:
points.distance(poly.iloc[0]) 

### 3.2.3 Spatial joining

In [ ]:
np.random.seed(11)       ## set seed for reproducibility
bb = world.total_bounds  ## the world's bounds
x = np.random.uniform(low=bb[0], high=bb[2], size=10)
y = np.random.uniform(low=bb[1], high=bb[3], size=10)
random_points = gpd.points_from_xy(x, y, crs=4326)
random_points = gpd.GeoDataFrame({'geometry': random_points})
random_points.plot(color='red', markersize=10)

In [ ]:
world_random = world[world.intersects(random_points.union_all())]
world_random

In [ ]:
random_joined = random_points.sjoin(world_random, how='left')
random_joined

In [ ]:
# Random points
base = world.plot(color='white', edgecolor='lightgrey')
random_points.plot(ax=base, color='None', edgecolor='red');
# World countries intersecting with the points
base = world.plot(color='white', edgecolor='lightgrey')
world_random.plot(ax=base, column='name_long');
# Points with joined country names
base = world.plot(color='white', edgecolor='lightgrey')
random_joined.geometry.plot(ax=base, color='grey')
random_joined.plot(ax=base, column='name_long', legend=True);

### 3.2.4 Non-overlapping joins

In [ ]:
cycle_hire.explore(
    tiles='Esri WorldImagery',
    color = 'blue',
    marker_kwds = {'radius': 5}
)

In [ ]:
cycle_hire_osm.explore(
    tiles='Esri WorldImagery',
    color = 'red',
    marker_kwds = {'radius': 5}
)

In [ ]:
base = cycle_hire.plot(
    edgecolor = 'blue', color='none')
cycle_hire_osm.plot(ax=base, edgecolor = 'red', color='none')

In [ ]:
print(cycle_hire_osm.geometry)
print(cycle_hire_osm.geometry.iloc[0])

In [ ]:
m = cycle_hire.geometry.apply(
    lambda x: cycle_hire_osm.geometry.intersects(x)
)
print(m)
m.to_numpy().any()

In [ ]:
cycle_hire

In [ ]:
crs = 27700
cycle_hire_buffers = cycle_hire.copy().to_crs(crs)
cycle_hire_buffers['geometry'] = cycle_hire_buffers.buffer(20)
cycle_hire_buffers = gpd.sjoin(
    cycle_hire_buffers,
    cycle_hire_osm.to_crs(crs),
    how='left',
    predicate='intersects'
)
cycle_hire_buffers

In [ ]:
cycle_hire_buffers = cycle_hire_buffers[['id', 'capacity', 'geometry']] \
                                         .dissolve(by='id', aggfunc='mean') \
                                         .reset_index()
cycle_hire_buffers.geometry = cycle_hire_buffers.centroid
cycle_hire_buffers

In [ ]:
cycle_hire_buffers.plot()

In [ ]:
# input
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
cycle_hire_osm.plot(ax=ax, column='capacity', legend=True, markersize=5)

In [ ]:
# join result
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
cycle_hire_buffers.plot(ax=ax, column='capacity', legend=True, markersize=5)

### 3.2.5 Spatial aggregation

In [ ]:
nz_height2 = gpd.sjoin(
    nz_height[['elevation', 'geometry']],
    nz[['Name', 'geometry']],
    how='left',
    predicate='intersects'
)
nz_height2

In [ ]:
nz_height2 = nz_height2.groupby('Name')[['elevation']].mean().reset_index()
nz_height2

In [ ]:
nz2 = pd.merge(
    nz[['Name', 'geometry']],
    nz_height2,
    how='left',
    on='Name'
)
nz2

In [ ]:
nz2.plot(
    column='elevation',
    cmap='Blues',
    legend=True,
    edgecolor='black',
    missing_kwds={'color': "grey", 'edgecolor': 'black'}
)

### 3.2.6 Joining incongruent layers

In [ ]:
# Settings: grid extent, resolution, and CRS
bounds = nz.total_bounds
crs = nz.crs
res = 100000
# Calculating grid dimensions
xmin, ymin, xmax, ymax = bounds
cols = list(range(int(np.floor(xmin)), int(np.ceil(xmax+res)), res))
rows = list(range(int(np.floor(ymin)), int(np.ceil(ymax+res)), res))
rows.reverse()
# For each cell, create 'shapely' polygon (rectangle)
polygons = []
for x in cols:
    for y in rows:
        polygons.append(
            shapely.Polygon([(x,y), (x+res, y), (x+res, y-res), (x, y-res)])
        )
# To 'GeoDataFrame'
grid = gpd.GeoDataFrame({'geometry': polygons}, crs=crs)
# Remove rows/columns beyond the extent
sel = grid.intersects(shapely.box(*bounds))
grid = grid[sel]
# Add consecultive IDs
grid['id'] = grid.index
grid

In [ ]:
base = grid.plot(color='none', edgecolor='grey')
nz.plot(
    ax=base,
    column='Population',
    edgecolor='black',
    cmap='Reds',
    legend=True
)


In [ ]:
nz['area'] = nz.area
nz

In [ ]:
nz_grid = nz.overlay(grid)
nz_grid = nz_grid[['id', 'area', 'Population', 'geometry']]
nz_grid

In [ ]:
nz_grid.plot(color='none', edgecolor='black')

In [ ]:
nz_grid['area_sub'] = nz_grid.area
nz_grid

In [ ]:
base = grid.plot(color='none', edgecolor='grey')
nz_grid.plot(
    ax=base,
    column='area_sub',
    edgecolor='black',
    cmap='Reds',
    legend=True
)

In [ ]:
nz_grid['area_prop'] = nz_grid['area_sub'] / nz_grid['area']
nz_grid['population'] = nz_grid['Population'] * nz_grid['area_prop']
nz_grid

In [ ]:
nz_grid = nz_grid.groupby('id')['population'].sum().reset_index()
grid = pd.merge(
    grid,
    nz_grid[['id', 'population']],
    how='left',
    on='id'
)
grid

In [ ]:
base = grid.plot(
    column='population',
    edgecolor='black',
    cmap='Reds',
    legend=True
)
nz.plot(
    ax=base,
    color='none',
    edgecolor='grey',
    legend=True
)

In [ ]:
nz['Population'].sum()

In [ ]:
grid['population'].sum().round(0)

### 3.2.7 Distance relations

In [ ]:
nz_highest = nz_height.sort_values(by='elevation', ascending=False).iloc[0:3, :]
nz_highest

In [ ]:
canterbury_centroid = canterbury.centroid.iloc[0]

In [ ]:
nz_highest.distance(canterbury_centroid)

In [ ]:
sel = nz['Name'].str.contains('Canter|Otag')
co = nz[sel]
co

In [ ]:
d = nz_height.iloc[:3, :].apply(lambda x: co.distance(x.geometry), axis=1)
d

In [ ]:
fig, ax = plt.subplots(figsize=(8, 12))
co.plot(color='lightgrey', edgecolor='black', ax=ax)
co.apply(
    lambda x: ax.annotate(
        text=x['Name'],
        xy=x.geometry.centroid.coords[0],
        ha='center'),
    axis = 1
)
nz_height.iloc[:3, :].plot(color='none', edgecolor='black', ax=ax)